# Global Malnutrition Analysis

In this project, we have analyzed several sources of data relevant to malnutrition amongst children ranging from the ages of infant to 5 years old. The major components of our analysis is as follows: 

1. Extracting data from various sources including UNICEF, World Bank and more (listed below) 
2. Combining malnutrition data into a single dataframe with information regarding wasting, overweight and stunting conditions amongst the global population of children from 1978 to 2021 
3. Analysing countries with the highest prevalence of malnutrition 
4. Developing a world heat map visualization to see the global malnutrition trend over the years 
5. Assessing a relationship, if any, between COVID deaths experienced by the country and prevalence of malnutrition among children

## Data Sources

1. The malnutrtion data is from UNICEF.org and it has the ratio of kids under 5 with stunt/severe stunting, wasted/severe wasted, and overweight problems at country-level from 1970 to 2021.\
(datalink: https://data.unicef.org/resources/data_explorer/unicef_f/?ag=UNICEF&df=GLOBAL_DATAFLOW&ver=1.0&dq=.NT_ANT_HAZ_NE2+NT_ANT_HAZ_NE3..&startPeriod=2016&endPeriod=2021).

2. COVID deaths data from OurWorldInData.org.\
(datalink: https://ourworldindata.org/coronavirus-source-data)

3. Continent & Country Data from Kaggle.\
(datalink: https://www.kaggle.com/statchaitya/country-to-continent) 

## 1. Data Gathering

The data sources described above will be used to extract relevant data that we will use for our analysis.

In [3]:
# download necessary packages

# basic packages
import pandas as pd  #dataframe
import numpy as np  #calculation
import scipy as sp #calculation
# regression analysis
from sklearn import linear_model #linear regression
import statsmodels.api as sm #linear regression model
# graphing
import matplotlib.pyplot as plt #plot
import seaborn as sns #plot
from scipy.interpolate import interp1d #interpolation
from seaborn import heatmap #heatmap
# geopandas
import os
import geopandas as gpd
import descartes
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'descartes'

Download each of Waste/Severe Waste/Stunt/Severe Stunt/Overweight dataset and save in panda.DataFrames and show headlines for each dataframe.


In [ ]:
wasted_data = pd.read_csv("Wasted_Data.csv")
display(wasted_data[:3])

In [ ]:
# make sure the data downloaded is for wasting only
wasted_data['INDICATOR:Indicator'].unique() 

In [ ]:
ser_wasted_data=pd.read_csv("Severe_Wasted_Data.csv")
display(ser_wasted_data[:1])

In [ ]:
ser_wasted_data['INDICATOR:Indicator'].unique() #for severe wasting only

In [ ]:
stunted_data = pd.read_csv("Stunt_Data.csv")
display(stunted_data[:1])

In [ ]:
stunted_data['INDICATOR:Indicator'].unique() #stunting only

In [ ]:
ser_stunted_data = pd.read_csv("Severe_Stunt_Data.csv")
display(ser_stunted_data[:1])

In [ ]:
ser_stunted_data['INDICATOR:Indicator'].unique() #severe stunting only

In [ ]:
overweight_data = pd.read_csv("Overweight_Data.csv")
display(overweight_data[:1])

In [ ]:
overweight_data['INDICATOR:Indicator'].unique() #overweight only

## 2. Data Cleaning

We will now clean up the datasets by renaming columns for simplifcation and combining the datasets to make a concise dataframe for analysis. After the data cleaning phase, the clean dataframe should contain data for overweight, stunting and wasting among children as well as a mean that summarizes these percentages. 

In [ ]:
# rename columns and only keep the relevant data columns
rename_dfs=[wasted_data,ser_wasted_data,stunted_data,ser_stunted_data,overweight_data]
rename_names=['Waste_percent','Severe_Waste_percent','Stunt_percent','Severe_Stunt_percent','Overweight_percent']
for i in range(len(rename_dfs)):
    rename_dfs[i]=rename_dfs[i].rename(columns = {"REF_AREA:Geographic area" : "Country", "SEX:Sex" : "Sex",\
                                     "TIME_PERIOD:Time period" : "Year", "OBS_VALUE:Observation Value" :rename_names[i] ,\
                                           'AGE:Current age':'Age'})
    rename_dfs[i]=rename_dfs[i][['Country','Sex','Year',rename_names[i],'Age']]
    
    

In [ ]:
# showing a sample of dataframes to make sure the names are succesfully changed
display(rename_dfs[0].columns)

In [ ]:
# Combine dataframes on columns except the malnutrition percents
combine_cols=rename_dfs[0].columns.tolist()
combine_cols.pop(3)
combined_df=wasted_df.merge(ser_wasted_df,how='outer',on=combine_cols)\
            .merge(stunted_df,how='outer',on=combine_cols)\
            .merge(ser_stunted_df,how='outer',on=combine_cols)\
            .merge(overweight_df,how='outer',on=combine_cols)\
# shown sample lines
combined_df.head(3)

In [ ]:
# Calculate total waste percent column from adding percentage with normal waste condition and percentage with severe waste condition
combined_df['Waste_total_percent']=combined_df['Waste_percent']+combined_df['Severe_Waste_percent']
combined_df['Stunt_total_percent']=combined_df['Stunt_percent']+combined_df['Severe_Stunt_percent']
combined_df.head(1)

In [ ]:
# pop the previous waste & stunt columns, only keep the "waste total" and "stunt total" column
cols=['Waste_percent','Severe_Waste_percent','Stunt_percent','Severe_Stunt_percent']
for c in cols:
    combined_df.pop(c)
combined_df.head(1)

In [ ]:
# save the column names for future uses
mal_type=['Mean','Overweight_percent','Waste_total_percent','Stunt_total_percent']
#malnutrition type 0=mean
#malnutrition type 1=overweight
#malnutrition type 2=waste
#malnutrition type 3=stunt

Now let's observe the properties of columns of our cleaned new combined dataframe.

In [ ]:
# 1. Make sure the age is all same.
combined_df['Age'].unique()
print('All data are for kids under 5 years old.')

In [ ]:
# Since the data only for kids under 5, we can delete this column.
del combined_df['Age']  

In [ ]:
# 2. Format sex column
combined_df['Sex'].unique()

In [ ]:
# only leave the "Total" for analysis
combined_df["Sex"] = combined_df["Sex"].str[1]
combined_df=combined_df[combined_df["Sex"]=='T']
del combined_df['Sex']  

In [ ]:
# 3. Format countries and regions
country_count=len(combined_df['Country'].unique())
print('There are a total number of '+ str(country_count) + ' countries/regions in the DataFrame initially.')

In [ ]:
# Format countries, separate to 3 digit ISO code and name
split_countries= combined_df["Country"].str.split(':', expand=True)

# join to combine df
combined_df = split_countries.join(combined_df)

# rename columns
combined_df = combined_df.rename(columns = {0 : "ISO Code", 1 : "Country Name"})

# format the region
combined_df["Country Name"] = combined_df["Country Name"].str.strip()

# filter only the countrys not regions
combined_df = combined_df[combined_df["ISO Code"].str.contains("_")==False]

# remove previous columns
del combined_df['Country']

combined_df.head(3)

In [ ]:
new_country_count=len(combined_df['Country Name'].unique())
print('There are now a total number of '+ str(new_country_count) + ' countries in the DataFrame now.')

Its important to note that there are 195 countries in the world indicating that certain countries are missing from this dataset. The data from these missing countries may significantly alter the course of the analysis but we continue to analysis the dataset as is. 

In [ ]:
# 4. Observe the Year columns
combined_df['Year'].unique()

Observe some data is recored per date instead of per year, and we only want to keep the dataframe one row per year, so we need to format the Year and keep the average of all percentages if there is more than one year appeared for each country.

In [ ]:
combined_df["Year"] =  pd.to_datetime(combined_df["Year"]).dt.year
combined_df=combined_df.groupby(['ISO Code','Country Name','Year'],as_index=False).mean()
display(combined_df[:3])

In [ ]:
#observe year ranges
years=combined_df['Year'].unique()
print(sorted(years))

In [ ]:
# 5. Now our dataframe is cleaned and we are ready to drop rows with Na values
combined_df.dropna(inplace=True) #drop NaN

In [ ]:
# create a new column of malnutrition score for mean of the five malnutrition factor
combined_df['Mean']=combined_df.iloc[:,4:9].mean(axis=1)
combined_df.head(3)

Its important to note here that the calculated mean of the various malnutrition conditions may not be the most ideal representation of each country during that year but we've chosen this method as a relatively high level status of the country. 

## 3. Creating Visualizations

In [ ]:
# now observe overweight/waste/stunt 
fig, axis = plt.subplots(2,2) 
fig.suptitle("World's Children Malnutrition Histogram")

# Mean
axis[0,0].hist(combined_df[mal_type[0]],bins=30,color='deepskyblue',alpha=0.5)
#axis[0,0].set_title("Mean malnutrition")
axis[0,0].set_xlabel('Mean malnutrition Percentage')
axis[0,0].set_ylabel('Number of Countries')

# Overweight
axis[0,1].hist(combined_df[mal_type[1]],bins=30,color='royalblue',alpha=0.5)
#axis[0,1].set_title("Overweight")
axis[0,1].set_xlabel('Overweight Percentage')
axis[0,1].set_ylabel('Number of Countries')

# Waste
axis[1,0].hist(combined_df[mal_type[2]],bins=30,color='orange',alpha=0.5)
#axis[1,0].set_title("Waste")
axis[1,0].set_xlabel('Waste Percentage')
axis[1,0].set_ylabel('Number of Countries')

# Stunt
axis[1,1].hist(combined_df[mal_type[3]],bins=30,color='forestgreen',alpha=0.5)
#axis[1,1].set_title("Stunt")
axis[1,1].set_xlabel('Stunt Percentage')
axis[1,1].set_ylabel('Number of Countries')


fig.set_size_inches(15, 10)
plt.show()

In [ ]:
plt.hist(combined_df[mal_type[0]],bins=30,color='deepskyblue',alpha=0.5)
plt.hist(combined_df[mal_type[1]],bins=30,color='royalblue',alpha=0.5)
plt.hist(combined_df[mal_type[2]],bins=30,color='orange',alpha=0.5)
plt.hist(combined_df[mal_type[3]],bins=30,color='forestgreen',alpha=0.5)
plt.xlabel('Percentage')
plt.ylabel('Number of countries')
plt.title('Combined Global Malnutrition Analysis Histogram')
fig.set_size_inches(15, 10)
plt.show()

In [ ]:
# now observe overweight/waste/stunt 
fig, axis2 = plt.subplots(2,2) 
fig.suptitle("World's Children Malnutrition Boxplot")

# Mean
axis2[0,0].boxplot(combined_df[mal_type[0]])
axis2[0,0].set_title("Mean malnutrition Percentage")

# Overweight
axis2[0,1].boxplot(combined_df[mal_type[1]])
axis2[0,1].set_title("Overweight Percentage")

# Waste
axis2[1,0].boxplot(combined_df[mal_type[2]])
axis2[1,0].set_title("Waste Percentage")

# Stunt
axis2[1,1].boxplot(combined_df[mal_type[3]])
axis2[1,1].set_title("Stunt Percentage")

fig.set_size_inches(15, 10)
plt.show()


In [ ]:
# observe correlation map
corr = combined_df.corr() # train is a pandas dataframe 
plt.figure(figsize = (8, 8))
heatmap(corr, cmap = 'Blues',annot=True)
plt.title('Correlation heatmap for all malnutrition types')
plt.show()

The above heatmap shows the following:
1. The relationship between the Mean and the Stunt percentage is relatively linear, which means stunt has been the most serious global malnutrition problem.
2. An obvious note, the correlation between overweight and stunting & wasting are negative, indicating negative correlation. This makes sense since it can be largely assumed that the factors that contribute to stunting and wasting would not be the same factors that contribute to overweight. 
3. It's evident that the negative correlation between the years and each of the conditions indicate that as the years have progressed, the percentage of malnutrition is decreasing over the years, the world has been improving!

In [ ]:
# Now Observe US first
country_df=combined_df.copy()
country_df.index = country_df["ISO Code"] #make the iso code as index 

country_df.loc['USA'].plot(x='Year', y=['Overweight_percent','Waste_total_percent','Stunt_total_percent'])
plt.title('US Children Malnutrition')
plt.xlabel('Year') # x label
plt.ylabel('Percentage') # y label
plt.grid(which='major', axis='both') # creating dash
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
fig.set_size_inches(10, 10)
plt.show()


Using interpolation to smooth the linear graph above so we will able to to view more continuous and time-based trends. 

In [ ]:
# create a function to smooth the graph later
# use interpld from scipy to smooth the lines

def countryGraph(countryISO):
    countryData=country_df.loc[countryISO]
    country_name=countryData['Country Name'][1]
    x=countryData['Year'] # x is the year
    X_=np.linspace(x.min(), x.max(),300) #X_ is the new x with linspace
    d=countryData[mal_type[1:]] #z is actual datapoints
    for i in range(1,4): 
        y=countryData[mal_type[i]] #interate the plot each y as malnutrtion types
        smooth_model = interp1d(x, y, kind = "cubic") #smooth with interplation
        f=smooth_model(X_) #f is the new y 
        plt.plot(X_,f) #plot
    plt.plot(x, d, 'ro') # red dots are the actual data points
    plt.plot(x, d, 'y--') # yellow dash lines are the linear plot
    plt.grid(which='major', axis='both') # creating dash
    plt.xlabel('Year') # x label
    plt.ylabel('Percentage') # y label
    plt.title(country_name+str(' Children Malnutrition Chart')) # title
    plt.legend(mal_type[1:],loc='center left', bbox_to_anchor=(1, 0.5)) # legend
    plt.show()
    
# Let's observe US data again now.
countryGraph('USA')


Observed that children under 5 in US have a relative high overweight problem, lower stunt problem, and the lowest waste problem.

Now let's groupby countries and observe the top 5 and bottom 5 countris with the highest and loweest overall mean malnutrition values. Because the minimum required datapoint of cubic spline is 4, we want to filter the country with 4 or more years of report.


In [ ]:
country_count_all=combined_df.groupby(by=['ISO Code','Country Name'],as_index=False).count()
country_count_all.rename(columns={'Mean':'Count'},inplace=True)
country_count_all=country_count_all[['ISO Code','Count']]

# filter country with >= 4 data points
country_count=country_count_all[country_count_all['Count']>=4]
country_count=country_count[['ISO Code','Count']]

# this is all countries
print('There are '+str(len(country_count_all))+' countries total.')

# this is all countries with 4+ data points 
print('There are '+str(len(country_count))+' countries with 4+ data points.')


In [ ]:
# now we are creating a country_rank dataframe to observe countries with 4+ data points in the rank of their malnutrition means.
country_rank=combined_df.groupby(by=['ISO Code','Country Name'],as_index=False).mean()
country_rank=country_rank.merge(country_count,how='inner',on='ISO Code')
country_rank=country_rank.sort_values(by='Mean')
country_rank["Year"]=country_rank["Year"].astype(int)
display(country_rank[:3])


In [ ]:
clean=country_rank.groupby(["Country Name", "Year"],as_index = False).mean()
iso_codes = combined_df[["ISO Code", "Country Name"]].drop_duplicates()
clean_rank = iso_codes.merge(clean, on = "Country Name")
clean_rank = clean_rank.sort_values(by = ["Mean"], ascending = True)
display(clean_rank[:3])

In [ ]:
# Save the 5 countries with lowest malnutrition mean for observation, save as top5 
top5=clean_rank[:5]['Country Name'].tolist()
display(top5)
# The 5 countries with least amount of children malnutrition problem

In [ ]:
# Save these 5 countries' corresponding ISO code as top5ISO
top5ISO=clean_rank[:5]['ISO Code'].tolist()
display(top5ISO)


In [ ]:
# Plot graphs to observe the trend for each malnutrition problem at country level for top 5 countries
for i in top5ISO:
    countryGraph(i)


In [ ]:
# Save the 5 countries with most amount malnutrition mean for observation, save as bot5 
bot5=clean_rank[-5:]['Country Name'].tolist()
display(bot5) 


In [ ]:
# Save these 5 countries' corresponding ISO code as bot5ISO
bot5ISO=clean_rank[-5:]['ISO Code'].tolist()
display(bot5ISO)


In [ ]:
# Plot graphs to observe the trend for each malnutrition problem at country level for top 5 countries
for i in bot5ISO:
    countryGraph(i)


analysis+ observations

### Continent Analysis

In [ ]:
country_rank_all=combined_df.groupby(by=['ISO Code','Country Name'],as_index=False).mean()
country_rank_all=country_rank_all.merge(country_count_all,how='inner',on='ISO Code')
country_rank_all=country_rank_all.sort_values(by='Mean', ascending = False)
country_rank_all["Year"]=country_rank_all["Year"].astype(int)
display(country_rank_all[:3])

In [ ]:
clean_all=country_rank_all.groupby(["Country Name", "Year"],as_index = False).mean()

clean_rank_all = iso_codes.merge(clean_all, on = "Country Name")

clean_rank_all = clean_rank_all.sort_values(by = ["Mean"], ascending = False)
display(clean_rank_all[:3])

In [ ]:
continent = pd.read_csv("countryContinent.csv")

continent = continent[["country", "continent"]]

joined = clean_rank_all.merge(continent, left_on = "Country Name", right_on = "country")

joined = joined.drop(columns = "country")

In [ ]:
display(joined[:3])

In [ ]:
by_continent = joined.groupby(["Year", "continent"], as_index = False).mean()

display(by_continent[:3])

In [ ]:
continents=by_continent['continent'].unique()

for c in sorted(continents):
    dfc=by_continent[by_continent['continent']==c]
    xc=dfc['Year']
    Xc=np.linspace(xc.min(), xc.max(),300)
    yc=dfc['Mean']
    f1= interp1d(xc, yc, kind = "cubic")
    yc_=f1(Xc)

#Plotting the Graph
    plt.title("Global Child Average Malnutrition Chart")
    plt.legend(continents,loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(which='major', axis='both')
    plt.plot(Xc, yc_,'--')


In the Global Malnutrition Trend, the following can be observed:

1. All continents seem to experience a downward trend in malnutrition cases towards the last 2-3 years 
2. A rapid spike in cases can be observed during 2003-2004. The SARS epidemic also began around this time which may have exacerbated the conditions for children already suffering from malnutrition in continents such as Asia and Europe
3. Its also important to note that this particular graph also accounts for overweight conditions which, depending on the global context and poverty status. For example, children in impoverished households with access to carbs but not to other nutrients could gain weight due to financial hardship or inavailability while children in certain 1st world countries may have access and capability to purchase nutritional food but families may simply choose not to due to other reasons. 

In [ ]:
for c in sorted(continents):
    dfc=by_continent[by_continent['continent']==c]
    xc=dfc['Year']
    Xc=np.linspace(xc.min(), xc.max(),300)
    yc=dfc['Stunt_total_percent']
    f1= interp1d(xc, yc, kind = "cubic")
    yc_=f1(Xc)

#Plotting the Graph
    plt.title("Global Child Stunting Data Chart")
    plt.grid(which='major', axis='both')
    plt.plot(Xc, yc_)

In this Stunting dataset, the following can be observed: 

1. While Africa's stunting trend is upward, the other continents end upon an upward trend. 
2. Its interesting to know the relatively close trend followed by Asia and Europe. 

In [ ]:
for c in sorted(continents):
    dfc=by_continent[by_continent['continent']==c]
    xc=dfc['Year']
    Xc=np.linspace(xc.min(), xc.max(),300)
    yc=dfc['Overweight_percent']
    f1= interp1d(xc, yc, kind = "cubic")
    yc_=f1(Xc)

#Plotting the Graph
    plt.title("Global Child Overweight Data Chart")
    plt.legend(continents,loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(which='major', axis='both')
    plt.plot(Xc, yc_)

In this overweight dataset, the following can be observed: 

1. Africa's trend seems to have the most variation throughout the timespan. While the 2001 data point may be a potential outlier, the 2008 data point may be attributed to a historical event such as the food production crisis in Africa or potentially due to multiple countries being included in the dataset during this year. 
2. All the continents are on an upward trend for increasing overweight child population except for the Americas. Its also to be noted that the dataset does not reach the same end year. 

In [ ]:
for c in sorted(continents):
    dfc=by_continent[by_continent['continent']==c]
    xc=dfc['Year']
    Xc=np.linspace(xc.min(), xc.max(),300)
    yc=dfc['Waste_total_percent']
    f1= interp1d(xc, yc, kind = "cubic")
    yc_=f1(Xc)

#Plotting the Graph
    plt.title("Global Child Wasting Data Chart")
    plt.legend(continents,loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(which='major', axis='both')
    plt.plot(Xc, yc_)

In this Wasting dataset, the following can be observed: 
    
1. Europe looks to be experiencing the most drastic peaks in wasting percentage both around 2012 and 2003. 
2. Both Africa and the Americas observe a peak during 2008. Perhaps the 2008 peak for the Americas may have some connection with the housing crisis and following recession period. Its also important to note that the Americas is a big region not only attributed to the United States. 

## Mapping

In [ ]:
# import map
world_map = gpd.read_file(os.getcwd()+'/World_Map/World_Countries__Generalized_.shp')

In [ ]:
map_all = world_map.merge(combined_df, how='left', left_on='COUNTRYAFF', right_on='Country Name')
print('Verifying data has combined')
display(map_all.head(1))

In [ ]:
print('Overweight observations range', np.nanmin(map_all['Overweight_percent'].values), '% to', np.nanmax(map_all['Overweight_percent'].values), '%.')
print('Wasted observations range', np.nanmin(map_all['Waste_total_percent'].values), '% to', np.nanmax(map_all['Waste_total_percent'].values), '%.')
print('Stunted observations range', np.nanmin(map_all['Stunt_total_percent'].values), '% to', np.nanmax(map_all['Stunt_total_percent'].values), '%.')

In [ ]:
# map with all overweight indicators
vmin, vmax = 0, np.nanmax(map_all['Overweight_percent'].values)
coloro = 'PuBu'

figo, axo = plt.subplots(1, figsize = (16,12))
map_all.plot(column='Overweight_percent', cmap=coloro, ax=axo, linewidth=0.2, edgecolors='0.3')
bar_infoo = plt.cm.ScalarMappable(cmap=coloro, norm=plt.Normalize(vmin=vmin, vmax=vmax))
axo.set_title('Percent of overweight children under age 5 around the world', fontdict={'fontsize':20})
axo.set_axis_off()
bar_infoo._A = []
cbar = figo.colorbar(bar_infoo, orientation='horizontal')

In [ ]:
# map with all waste indicators
vmin, vmax = 0, np.nanmax(map_all['Waste_total_percent'].values)
colorw = 'YlOrRd'

figw, axw = plt.subplots(1, figsize = (16,12))
map_all.plot(column='Waste_total_percent', cmap=colorw, ax=axw, linewidth=0.2, edgecolors='0.3')
bar_infow = plt.cm.ScalarMappable(cmap=colorw, norm=plt.Normalize(vmin=vmin, vmax=vmax))
axw.set_title('Percent of wasted children under age 5 around the world', fontdict={'fontsize':20})
axw.set_axis_off()
bar_infow._A = []
cbar = figw.colorbar(bar_infow, orientation='horizontal')

In [ ]:
# map with all stunt indicators
vmin, vmax = 0, np.nanmax(map_all['Stunt_total_percent'].values)
colors = 'YlGn'

figs, axs = plt.subplots(1, figsize = (16,12))
map_all.plot(column='Stunt_total_percent', cmap=colors, ax=axs, linewidth=0.2, edgecolors='0.3')
bar_infos = plt.cm.ScalarMappable(cmap=colors, norm=plt.Normalize(vmin=vmin, vmax=vmax))
axs.set_title('Percent of stunted children under age 5 around the world', fontdict={'fontsize':20})
axs.set_axis_off()
bar_infos._A = []
cbar = figs.colorbar(bar_infos, orientation='horizontal')

## Animation

In [ ]:
# list of years
min_year = int(map_all['Year'].min())
max_year = int(map_all['Year'].max())+1
list_of_years = list(range(min_year, max_year))
print('Years range from', min(list_of_years), 'to', max(list_of_years))

In [ ]:
# list of countries
countries = map_all['COUNTRYAFF'].unique()
print('There are', len(countries), 'countries listed.')

In [ ]:
# save all the maps in the pictures folder
output_path = os.getcwd()+'/pictures'
filenames = []

In [ ]:
# algorithm to make a map for each year and save as jpg
def make_maps(map_col, color, ax, list_of_years = list(range(2000,2020))):
    # start the for loop to create one map per year
    for year in list_of_years:
        # narrow the data to the year desired
        yearly = combined_df.loc[combined_df['Year'] == year].reset_index(drop=True)
        maps = world_map.merge(yearly, how='left', left_on='COUNTRYAFF', right_on='Country Name')
        
        # normalize across all years
        vmin = min(combined_df[map_col])
        vmax = max(combined_df[map_col])

        # create map
        fig = maps.plot(column=map_col,
                        cmap=color,
                        figsize=(16,12),
                        linewidth=0.3,
                        edgecolor='0.3',
                        ax=ax,
                        norm=plt.Normalize(vmin=vmin, vmax=vmax))
        
        # add a title
        fig.set_title(map_col + ' children under age 5 in ' + str(year), \
                  fontdict={'fontsize': '20',
                             'fontweight' : '3'})

        # remove axis of chart
        fig.set_axis_off()
        
        # this will save the figure as a high-res jpg
        filename = str(year)+'_'+map_col+'.jpg'
        filepath = os.path.join(output_path, filename)
        filenames.append(filename)
        chart = fig.get_figure()
        chart.savefig(filepath, dpi=300)

In [ ]:
# make all the annual maps
make_maps('Overweight_percent', coloro, axo, list_of_years)

In [ ]:
make_maps('Waste_total_percent', colorw, axw, list_of_years)

In [ ]:
make_maps('Stunt_total_percent', colors, axs, list_of_years)

In [ ]:
# pull the created pictures into a gif
import glob
from PIL import Image

def make_gif(fp_in, fp_out, duration = 400, loop = 3):
    img, *imgs = [Image.open(f) for f in sorted(glob.glob(fp_in))]
    img.save(fp=fp_out, format='GIF', append_images=imgs,
             save_all=True, duration=duration, loop=loop)

In [ ]:
# filepaths for overweight gif
o_fp_in = os.getcwd()+'/pictures/*_Overweight*.jpg'
o_fp_out = os.getcwd()+'/overweight.gif'

#make overweight gif
make_gif(o_fp_in, o_fp_out)

In [ ]:
# filepaths for wasted
w_fp_in = os.getcwd()+'/pictures/*_Waste*.jpg'
w_fp_out = os.getcwd()+'/waste.gif'

#make wasted gif
make_gif(w_fp_in, w_fp_out)

In [ ]:
# filepaths for stunted
s_fp_in = os.getcwd()+'/pictures/*_Stunt*.jpg'
s_fp_out = os.getcwd()+'/stunt.gif'

#make stunted gif
make_gif(s_fp_in, s_fp_out)

![SegmentLocal](overweight.gif "segment")

![SegmentLocal](waste.gif "segment")

![SegmentLocal](stunt.gif "segment")

### Covid Analysis

In [ ]:
# combine with COVID dataset

In [ ]:
# Download covid vaccine rate data to covid_data from https://ourworldindata.org/covid-vaccinations
covid_data=pd.read_csv("owid-covid-data.csv")
covid_data.tail(5)

In [ ]:
#before 12/31/2020, the total number of cases for each countries
covid_data=covid_data[covid_data['date']<'2021-01-01']
covid_2020=covid_data.groupby(by='iso_code').max()
display(covid_2020[:5])


In [ ]:
norm_covid_df=covid_2020.copy()
# first drop continent= Na only leave the countries
norm_covid_df=norm_covid_df[norm_covid_df['continent'].notna()]
# second fill in nan with 0 for countries has no deaths
norm_covid_df['total_deaths']=norm_covid_df['total_deaths'].fillna(0)

# since cases number varies alot, apply z-score standardlization to total_cases column to normalize the case number
# norm_covid_df['norm_total_deaths'] = (norm_covid_df['total_deaths'] - norm_covid_df['total_deaths'].mean())/\
#                                     (norm_covid_df['total_deaths'].std())    

# apply max-min normalization so the range from 0 to 1
norm_covid_df['norm_total_deaths'] =norm_covid_df['total_deaths'] /norm_covid_df['total_deaths'].abs().max()

#view by sorting in normalized total death
norm_covid_df=norm_covid_df.sort_values(by='norm_total_deaths')
norm_covid_df.tail(5)



In [ ]:
# observe the histogram of normalized total death due to covid for all countries
plt.hist(norm_covid_df['norm_total_deaths'],bins=50,color='deepskyblue')
plt.xlabel('Normalized total death')
plt.ylabel('Number of Countries')
plt.show()


In [ ]:
norm_covid_2020['norm_total_deaths'].describe()

In [ ]:
# Let's observe the countries with over 75% percentail's covid death as serious countries
norm_covid_serious_df=norm_covid_df[norm_covid_df['norm_total_deaths']>=0.008371] 
display(norm_covid_serious_df[:5])
plt.hist(norm_covid_serious_df['norm_total_deaths'],bins=50,color='deepskyblue')
plt.show()


#### Highest % of COVID deaths in the world

In [ ]:
covid = pd.read_csv("owid-covid-data.csv")[["location", "date", "new_deaths"]]

covid["date"] = pd.to_datetime(covid["date"]).dt.year

top_10_covid = covid.groupby(["location", "date"], as_index = False).sum()

display(top_10_covid[:3])

In [ ]:
worldpop = pd.read_csv("Worldpop.csv")

worldpop["PopTotal"] = worldpop["PopTotal"].astype(int)

display(worldpop[:3])

Join total population to covid deaths for comparison

In [ ]:
top_10_covid = worldpop.merge(top_10_covid, left_on = ["Location", "Time"], right_on = ["location", "date"])

top_10_covid = top_10_covid.drop(columns = ["date", "location"])

In [ ]:
display(top_10_covid[:3])

In [ ]:
# population total in 1000's 

top_10_covid["deaths percentage"] = (top_10_covid["new_deaths"]/(top_10_covid["PopTotal"]*1000))

top_10_covid = top_10_covid[top_10_covid["Location"] != "World"]

covid_countries = top_10_covid.sort_values(by = "deaths percentage", ascending = False)

In [ ]:
covid_countries = covid_countries[:10]

display(covid_countries[:3])

In [ ]:
covid_trend = top_10_covid[top_10_covid["Location"].isin(covid_countries["Location"].unique())]

display(covid_trend[:3])

In [ ]:
covid_trend = top_10_covid[top_10_covid["Location"].isin(covid_countries["Location"].unique())]
covid_trend = covid_trend.sort_values(by = ["deaths percentage", "Location"], ascending = False)
display(covid_trend[:3])

In [ ]:
sns.barplot(x="Location", y="deaths percentage",
             hue="Time",data=covid_trend, palette="RdPu", lw =3).set(title = "Countries with highest Covid-19 % of Deaths")

sns.set(rc={'figure.figsize':(15,6)})

sns.set_style("white")

In [ ]:
# countries with most covid deaths 
continent = pd.read_csv("countryContinent.csv")

continent = continent[["country", "continent"]]

covid_deaths = covid_trend.merge(continent, left_on = "Location", right_on = "country")

covid_deaths = covid_deaths.drop(columns = ["country"])

display(covid_deaths["continent"].unique())

### PCA Compression

In [ ]:
new_analysis = combined_df.groupby(["Country Name"]).mean()

new_analysis = new_analysis.T

new_analysis = new_analysis.drop(["Year"], axis = 0)
new_analysis = new_analysis.drop(["Mean"], axis = 0)
new_analysis['Malnutrition Factor'] = new_analysis.index
new_analysis.reset_index(drop=True, inplace=True)
new_analysis.insert(0, 'Malnutrition Factor', new_analysis.pop("Malnutrition Factor"))
display(new_analysis)

In [ ]:
countries = new_analysis.columns.to_list()
countries.remove("Malnutrition Factor")

factors = new_analysis["Malnutrition Factor"]

In [ ]:
X_raw = new_analysis[countries].values.T

s = min(X_raw.shape)

X = X_raw - np.mean(X_raw, axis=0)

U, Sigma, VT = np.linalg.svd(X, full_matrices=False) # What does the `full_matrices` flag do?


In [ ]:
fig = plt.figure(figsize=(20, 20))
Y_k = X.dot(VT[0:10, :].T)

plt.scatter(Y_k[:, 0], Y_k[:, 2], c = "orange", s = 100)
plt.title("PCA Algorithm on Global Malnutrition")
for x, y, label in zip(Y_k[:, 0], Y_k[:, 2], countries):
    plt.annotate(label, xy=(x, y))

## Create a scoring system for the countries

Now we will create a scoring system for all countries to be able to see if there is a possible corellation of malnutrition on COVID-19 survival rate. The malnutrition score adds all 3 types of malnutrition measurements for a final total 'score'. The COVID-19 score is a function of total number of deaths divided by the length of time since the country first reported a death.

In [ ]:
# calculate a malnutrition score for each country
mal_score = combined_df.copy()
mal_score['mscore'] = mal_score['Overweight_percent']+mal_score['Waste_total_percent']+mal_score['Stunt_total_percent']
mal_score.drop(labels=['Overweight_percent', 'Waste_total_percent', 'Stunt_total_percent', 'Mean', 'Year'], axis=1, inplace=True)
mal_score = mal_score.groupby(by=['Country Name']).mean()
print('Malnutrition score range is from ', round(max(mal_score['mscore']),3), 'to', round(min(mal_score['mscore']),3))

In [ ]:
# normalize the score values
from sklearn.preprocessing import MinMaxScaler, normalize
scaler = MinMaxScaler(feature_range=(0,10))
ms = scaler.fit_transform(mal_score)
scaled_mal_score = pd.DataFrame(ms, index=mal_score.index, columns=mal_score.columns)
print('Now, malnutrition score range is from ', round(max(scaled_mal_score['mscore']),3), 'to', round(min(scaled_mal_score['mscore']),3))

In [ ]:
# filter the covid dataframe to only have the columns necessary
covid_score = covid_data.copy()
covid_score = covid_score[['location', 'date', 'total_deaths']]

# count how many times the country name occurs so we know how many dates they have had COVID-19
counts = pd.DataFrame(covid_score['location'].value_counts(dropna=False))

# find last known total number of deaths for each country
deaths = covid_score.groupby(by='location').max()

In [ ]:
# calculate the covid score for each country
cscore =  deaths.merge(counts, left_on='location', right_index=True)
cscore.rename(columns={'location': 'num_reports'}, inplace=True)
cscore['cscore'] = cscore['total_deaths'] / cscore['num_reports']
cscore.drop(labels=['date', 'total_deaths', 'num_reports'], axis=1, inplace=True)
cscore.fillna(0, inplace=True) # there were 7 countries with no covid score
print('Covid score range is from ', round(max(cscore['cscore']),3), 'to', round(min(cscore['cscore']),3))

In [ ]:
# normalize the score values
cs = scaler.fit_transform(cscore)
scaled_covid_score = pd.DataFrame(cs, index=cscore.index, columns=cscore.columns)
print('Now, covid score range is from ', round(max(scaled_covid_score['cscore']),3), 'to', round(min(scaled_covid_score['cscore']),3))

In [ ]:
# join the scores for comparison
merged_scores = scaled_mal_score.merge(scaled_covid_score, left_index=True, right_index=True)

In [ ]:
# now observe distribution
fig, axis2 = plt.subplots(1,2) 
fig.suptitle("Scores Distribution Boxplot", fontsize=20)

# Malnutrition score
axis2[0].boxplot(merged_scores['mscore'])
axis2[0].set_title("Malnutrition score distribution")

# COVID-19 score
axis2[1].boxplot(merged_scores['cscore'])
axis2[1].set_title("COVID-19 score distribution")

In [ ]:
# scatterplot of scores to look for correlation
x1 = merged_scores['mscore']
y1 = merged_scores['cscore']

plt.scatter(x1, y1, color='royalblue', alpha=0.3)
plt.title('COVID-19 score vs. Malnutrition score', fontsize=20)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel("Malnutrition score")
plt.ylabel("COVID-19 score")

m1, b1 = np.polyfit(x1, y1, 1)
plt.grid(which='major', axis='both')
plt.plot(x1, m1*x1 +b1, 'royalblue')



Creating an interaction term is a measurable way to see the effect of malnutrition on COVID-19 deathrate based on the scores we just created and assigned to each country. This interaction term is a multiple of the malnutrition score times the COVID-19 score.

In [ ]:
# now create an interaction term for easy comparison
merged_scores['interaction_term'] = merged_scores['mscore'] * merged_scores['cscore']
merged_scores = merged_scores.sort_values(by=['interaction_term'], ascending=False)
display('Top 5 countries where COVID-19 has a large affect compared to malnutrition', merged_scores[:5])#, 'Bottom 5', merged_scores[-5:])

In [ ]:
plt.plot(merged_scores['interaction_term'],merged_scores.index)
plt.title('Distribution of Interaction Term', fontsize=20)
plt.ylabel('Countries')
plt.show()

In [ ]:
# isolate the outliers
outliers = merged_scores[merged_scores.interaction_term >= 0.5] #.tolist()
display(outliers)

In [ ]:
# bubble plot
plt.figure(figsize=(15, 8))
plt.scatter(outliers['mscore'], outliers['cscore'],
            color = 'purple',  # can also be c=outliers['interaction_term']
            alpha=0.5,
            s = outliers['interaction_term'] * 2000)
plt.title('Coutries with highest correlation COVID-19 score vs. Malnutrition score', fontsize=20)
plt.xlabel('Malnutrition score', size=14)
plt.ylabel('COVID-19 score', size=14)

for i, txt in enumerate(list(outliers.index)):
    plt.annotate(txt, (outliers['mscore'][i], outliers['cscore'][i]))

## About the authors:

The authors met as students in Georgia Institue of Technology's Master of Science in Analytics program.  They were in a study group together for the Introduction for Computing for Data Analytics (CSE 6040) class in Fall 2021.

#### Yuxin Lui

https://www.linkedin.com/in/yuxin-liu-97108bb0/

#### Rutuja Patil

https://www.linkedin.com/in/rutuja-patil-b2804413b/

#### Kimberly Austin

https://www.linkedin.com/in/kimberly-austin-ga/

Modified: 06 Jan 2022